<center><img src="../img/ICHEC_Logo.png" alt="Drawing" style="width: 500px;"/>



# <center> Timing Code & Simple Speed-up Techniques

******
***
## <center> <b>Performance Measurement</b>
    
Before diving into methods to speed up a code, we need to look at some tools which can help us understand how long our code actually takes to run.

Performance code profiling is a tool used to identify and analyse the execution of applications, and identify the bits of code that can be improved to achieve a better speed of the calculations or a  better flow of information and memory usage.

This form of dynamic program analysis can provide information on several aspects of program optimization, such as:
<details>
    <summary markdown="span"><b></b></summary>
   - How long a method/routine takes to execute?<br>
   - How often a routine is called? <br>
   - How are memory allocations and garbage collections tracked<br>
   - How often web services are called?
<br>
</details>

It is important to **never try and optimise your code on the first try**. Get the code correct first. It is often said that *premature optimization is the root of all evil* when it comes to programming.

Before optimising a code, one needs to find out where the time is spent most. Around 90% of the time is spent in 10% of the application.

<details>
    <summary markdown="span">There are a number of different methods</summary>
   <br>
   - <code>time</code> module<br>
   - <code>timeit</code> module<br>
   - <code>cProfile</code> module<br>
   - <code>datetime</code> module <br>
   - <code>astropy</code> module (mainly used in astronomy)<br>
   - Full fledged profiling tools: TAU, Intel Vtune, Python Tools for Visual Studio ...
<br>
</details>

***
### [`time`](https://docs.python.org/3/library/time.html)</code> Module</h2>

Python's `time` module can be used for measuring time spent in specific part of the program
* `time.time()`
    - Absolute time; real-world time measured from a fixed point in the past.
* In Python 3: `time.perf_counter()`, `time.process_time()`
    - Relative time; unit-less value which is proportional to the time elapsed between two instants.

In [1]:
import time

print (time.time())

print (time.perf_counter()) # includes time elapsed during sleep (CPU counter)
print (time.process_time()) # does not include time elapsed during sleep

1652264171.399796
2.433484803
0.885057


In [2]:
t0 = time.time()

my_list = []
for i in range(500): 
    my_list.append(0)
    
t1 = time.time()

tf = t1-t0

print('Time taken in for loop: ', tf)

Time taken in for loop:  0.00011992454528808594


***
### [`timeit`](https://docs.python.org/2/library/timeit.html) Module</h2>

* Easy timing of small bits of Python code.
* Tries to avoid common pitfalls in measuring execution times.
* Works best in command line interface

```
$ python -m timeit -s "from my module import func" "func()"

10 loops, best of 3: 433 msec per loop
```

* Python interface (<code>%timeit</code> magic in iPython)

```ipython
In [1]: from mymodule import func
In [2]: %timeit func()
```
```
10 loops, best of 3: 433 msec per loop
```

In [3]:
import random as rnd
import time
import math


def montecarlo_py(n):
    x, y = [], []
    for i in range(n):
        x.append(rnd.random())
        y.append(rnd.random())
    pi_xy = [(x[i], y[i]) for i in range(n) if math.sqrt(x[i] ** 2 + y[i] ** 2) <= 1]
    return(4 * len(pi_xy) / len(x))
    # Estimate for pi

In [4]:
montecarlo_py(1000000)

3.1417

In [5]:
%timeit montecarlo_py(1000000)

724 ms ± 6.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Another method is to use a setup like below

In [6]:
import timeit

setup_code = "from __main__ import montecarlo_py"
stmt = "montecarlo_py(1000000)"
times = timeit.repeat(setup=setup_code, stmt=stmt, repeat=3, number=10)
print(f"Minimum execution time: {min(times)}")

Minimum execution time: 7.153229266


****
## [`cProfile`](https://docs.python.org/2/library/profile.html#module-cProfile)

* cProfile provides an API for profiling your Python program
    * A profile is a set of stats
    * Time spent in different parts of the program

```python
# profile statement and save results to a file func.prof
cProfile.run('func()', 'func.prof')
```

In [7]:
import cProfile
cProfile.run('montecarlo_py(1000000)')

         5000007 function calls in 1.391 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.495    0.495    0.558    0.558 <ipython-input-3-7987cc796f5a>:11(<listcomp>)
        1    0.501    0.501    1.338    1.338 <ipython-input-3-7987cc796f5a>:6(montecarlo_py)
        1    0.053    0.053    1.391    1.391 <string>:1(<module>)
        1    0.000    0.000    1.391    1.391 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.len}
  1000000    0.063    0.000    0.063    0.000 {built-in method math.sqrt}
  2000000    0.128    0.000    0.128    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
  2000000    0.150    0.000    0.150    0.000 {method 'random' of '_random.Random' objects}




This can be modified to save the output to a `.prof` file.
```python
cProfile.run('func()', 'func.prof')
```
But there is not much point of this for a small function like this which has a limited runtime. We will look at another file, and this time run it through the terminal, and generate the `.prof` file.

In [8]:
!python -m cProfile -o heat_equation_simple.prof heat_equation_simple.py

Running Time: 20.20987105369568


We need a separate module to look at the profile we have just created.

***
### Investigating Profile with [`pstats`](https://docs.python.org/3/library/profile.html#module-pstats)

* Prints execution time of selected functions. 
* Sorts by function name, time, cumulative time, ... 
* Python module interface and interactive browser

In [9]:
from pstats import Stats
p = Stats('heat_equation_simple.prof')

p.strip_dirs() #The strip_dirs() method removes the extraneous path from all the module names

# Other string options include 'cumulative', 'name', 'ncalls'
p.sort_stats('time').print_stats(10)

Wed May 11 11:17:04 2022    heat_equation_simple.prof

         1007265 function calls (988402 primitive calls) in 22.701 seconds

   Ordered by: internal time
   List reduced from 5896 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      200   20.209    0.101   20.209    0.101 evolve.py:1(evolve)
      797    0.841    0.001    0.841    0.001 {built-in method io.open_code}
    68/66    0.178    0.003    0.183    0.003 {built-in method _imp.create_dynamic}
        1    0.135    0.135    0.135    0.135 {built-in method mkl._py_mkl_service.get_version}
      797    0.128    0.000    0.128    0.000 {built-in method marshal.loads}
      797    0.118    0.000    0.118    0.000 {method 'read' of '_io.BufferedReader' objects}
     3692    0.059    0.000    0.059    0.000 {built-in method posix.stat}
      797    0.043    0.000    1.002    0.001 <frozen importlib._bootstrap_external>:969(get_data)
 2528/979    0.038    0.000    1.107    0.

Using this for longer programs and more functions can help you pin down the functions in your code which need optimisation.

#### Using `pstats` in the terminal

```
$ python -m pstats myprof.prof
Welcome to the profile statistics
% strip
% sort time
% stats 5

Mon May 11 09:28:00 2020 my.prof
...
```

****
# <center> [Exercise](./01-Exercise-Fundamentals.ipynb)

****
****
<center><img src="../img/NumPy_Logo.png" alt="Drawing" style="width: 300px;"/>

    
## <center> <b>NumPy - Fast Array Interface</b>

### Python Lists

One of the 4 main data types in Python:
- lists `[1,2,3]`
- tuples `(1,2,3)`
- dictionaries `{"Food" : "fruit", "Type" : "banana"}`
- sets `{"apple", "banana"}`

![<Diagram 1>](../img/1_pylist.png)

Lists are ordered, changeable, start at index `[0]`, and can store all types of variables, including other data types.

In [10]:
# Flexiblilty of standard python array
a = [1, 2.1, '3', (4, 5.2), [6, {7.3 : '8'}, 9]]
a

[1, 2.1, '3', (4, 5.2), [6, {7.3: '8'}, 9]]

<details>
    <summary markdown="span"><b>Features of the Python List</b></summary>
<br>
   - Dynamic, elements of different types, including other types of 'array'<br>
   - Multiple dimensions<br>
   - Subarrays can have different number of elements<br>
<br>
<details>
    <summary markdown="span"><b>In short they are very flexible but not good in practice, why?</b></summary>
<br>
- Python has the luxury of being programmable without the user caring about the data types they are using. 
<br>
- Lists may be flexible but also slow to process in numerical computations.
<br>
- These are arrays of pointers to objects in sparse locations in memory, which cannot be easily cached and thus reading its values becomes a slower task.
</details>
</details>
 

<div class="alert alert-block alert-info">
<b>To become efficient in Data-driven programming and computation requires a good understanding of how data is stored and manipulated. This will help you in the long run. In statically typed languages like C++ or Java, all the variables have to be declared explicitly, a dynamically typed language like Python skips this step and is why it is more popular. This does have drawbacks when it comes to performance</b></div>

Thankfully, simple libraries like NumPy can assist with this

****
## NumPy arrays
<br>
<details>
    <summary markdown="span"><b>Features of the NumPy array</b></summary>
<br>
   - Provides a convenient interface for working with multi-dimensional array data structures efficiently.<br>
   - Arrays use contiguous blocks of memory that can be effectively cached by the CPU. <br>
   - NumPy sacrifices Python's flexibility to achieve low-memory usage and speed-up, as NumPy arrays have a fixed size and the datatype of its element must be homogeneous.<br>
   - Written in C, which is known for being a efficient programming language in terms of speed and memory usage.<br>
<br>
</details>
<br>

![<Diagram 2>](../img/1_numpy_arr.png)

NumPy arrays can be created from a Python list or tuple by using NumPy’s `array` function. The dimensionaility and shape of the resulting array will be determined by the given input. NumPy offers several functions for creating arrays, depending on the desired content and shape of the array.

When creating an array, NumPy will try to convert entries to convinient data type. If it is not possible, it will raise an error.

Link to Numpy documentation: [`array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html?highlight=array#numpy.array)

In [11]:
import numpy as np
# Creation from Python list
a = np.array((1,'2',3,4), float)

# Creation of 3D numpy array
b = np.array([[[1, 2.2],[3, 4]],[[5,6.6],[7.8, 9]]])

print("a: NumPy array from Python list\n", a, "\n")
print("b: 3D NumPy array\n", b)

a: NumPy array from Python list
 [1. 2. 3. 4.] 

b: 3D NumPy array
 [[[1.  2.2]
  [3.  4. ]]

 [[5.  6.6]
  [7.8 9. ]]]


<br>
<details>
    <summary markdown="span"><b>The features of the above NumPy array:</b></summary>
<br>
- Can have multiple dimensions<br>
- All elements have the same type<br>
- Number of elements in the array is fixed<br>
<br>
</details>

Understanding the shape and size of arrays is crucial in applications such as machine deep learning. You will often need to reshape arrays.

In [12]:
# Determine shape of array b
print("Array b:\n", b,)
print("\nShape:\n", b.shape)
print("\nSize:\n", b.size, "\n")

# Reshape b into a 1x8 matrix
b_new = b.reshape(1,8)
print("Array b_new:\n", b_new, b_new.shape, b_new.size)

Array b:
 [[[1.  2.2]
  [3.  4. ]]

 [[5.  6.6]
  [7.8 9. ]]]

Shape:
 (2, 2, 2)

Size:
 8 

Array b_new:
 [[1.  2.2 3.  4.  5.  6.6 7.8 9. ]] (1, 8) 8


This is fine, but creates an issue with storage, particularly for large arrays. As we no longer need array `b`, we can get rid of it with `del`.

In [13]:
del b

In [14]:
print(b)

NameError: name 'b' is not defined

In statically typed languages, one should always free up the memory taken up by variables, usually with keywords such as `free` 

***
### NumPy Array Indexing

Indexing and Slicing are the most important concepts in working with arrays.

**Indexing**

In [15]:
mat1d = np.array([1,2,3,4,5,6])
mat1d[1]

2

In [16]:
mat2d = np.array([[1,2,3], [4,5,6]])
print(mat2d[0,2], mat2d[:1])

3 [[1 2 3]]


In [17]:
mat3d = np.random.randint(1,50,(4,2,4))
print(mat3d)
mat3d[3,1,2]

[[[ 1 14 31 28]
  [ 2 32 38 33]]

 [[22 29 35  5]
  [47 38 10 37]]

 [[44  7 49 34]
  [ 7 15  2 45]]

 [[15 27 37 11]
  [38 40 34  7]]]


34

**Slicing**

In [18]:
mat1d = np.arange(10)
print(mat1d)
mat1d[3:]

[0 1 2 3 4 5 6 7 8 9]


array([3, 4, 5, 6, 7, 8, 9])

In [19]:
mat1d[:-2]

array([0, 1, 2, 3, 4, 5, 6, 7])

In [20]:
mat1d[1:7:2]

array([1, 3, 5])

In [21]:
mat3d = np.random.randint(1,10,(4,3,4))
print(mat3d)


[[[7 1 6 6]
  [5 5 9 2]
  [2 1 3 3]]

 [[4 6 7 3]
  [3 5 1 3]
  [2 5 2 9]]

 [[7 2 5 3]
  [9 3 7 9]
  [9 1 2 4]]

 [[7 1 4 4]
  [3 5 9 4]
  [5 3 7 9]]]


In [22]:
mat3d[0:3, 0:2, 0:4:2] = 99
mat3d

array([[[99,  1, 99,  6],
        [99,  5, 99,  2],
        [ 2,  1,  3,  3]],

       [[99,  6, 99,  3],
        [99,  5, 99,  3],
        [ 2,  5,  2,  9]],

       [[99,  2, 99,  3],
        [99,  3, 99,  9],
        [ 9,  1,  2,  4]],

       [[ 7,  1,  4,  4],
        [ 3,  5,  9,  4],
        [ 5,  3,  7,  9]]])

* There are many possible ways of arranging items of $N-$dimensional array in a 1-dimensional block.
* NumPy uses striding where $N-$dimensional index; $ n_0, n_1, ... n_{(N-1)} $ corresponds to offset from the beginning of 1-dimensional block.

<img src="../img/1.3.2.png" alt="Drawing" style="width: 500px;"/>

<center><code>a = np.array(..)</code></center>
    
* `a.flags` - Various information about memory layout.
* `a.strides` - Bytes to step in each dimension when traversing. 
* `a.itemsize` - Size of one array element in bytes.
* `a.data` - Python buffer object pointing to start of arrays data. 
* `a.__array_interface__` - Python internal interface.

This should be familiar, so feel free to check out the NumPy documentation for more utilisation of functions

***
### Vectorisation

<details>
    <summary markdown="span"><b>Why use vectorisation?</b></summary>
<br>
    - <code>for</code> loops in Python are very slow<br>
    - vectorisation is an example of a SIMD operation<br>
    - one instruction carries out many operands in parallel<br>
    - less overhead compared to <code>for</code> loops<br>
</details>
<br>
Lets look at a difference example

In [23]:
def loop_it(n):
    t0 = time.time()
    arr = np.arange(n)
    dif = np.zeros(n-1, int)
    for i in range(1, len(arr)):
        dif[i-1] = arr[i] - arr[i-1]
    t1 = time.time()
    print('Loop version: {} seconds'.format(t1-t0))
    return dif
    
def vectorise_it(n):
    t0 = time.time()
    arr = np.arange(n)
    dif = arr[1:] - arr[:-1]
    t1 = time.time()
    print('Vectorised version: {} seconds'.format(t1-t0))
    return dif

In [24]:
n=10000000
loop_it(n)
vectorise_it(n)

Loop version: 4.1977081298828125 seconds
Vectorised version: 0.02220916748046875 seconds


array([1, 1, 1, ..., 1, 1, 1])

***
# <center> [Exercise](./01-Exercise-Fundamentals.ipynb)

***
***
## <center> <b>Caching</b>

### What is the cache?

The cache is a part of the computer's memory. Caching can provide an application performance boost as it is faster to access data from the temporary location than it is to fetch the data from the source each time.
<br>
<br>
<details>
    <summary markdown="span"><b>A computer's memory can consists of three elements</b></summary>
<br>
   - Main memory (RAM or ROM): <br>
   - Secondary memory<br>
   - Cache: Acts as a buffer betwwen the CPU and main memory, used to hold parts of data and program most frequently used by the CPU.<br>
<br>
</details>
<br>
<details>
    <summary markdown="span"><b>The main rules of caching:</b></summary>
<br>
   - If a function is frequently called, its output is not changing often and it takes a long time to execute, it is a suitable candidate to implement caching. <br>
   - Caching should be faster than getting the data from the current data source<br>
   - Caching impacts memory footprint, so it is crucial to choose and cache the data structures and attributes that need to be cached.<br>
<br>
</details>


***
### What is caching?

Caching itself is an optimization strategy that you can use in your applications to keep recent or often used data in memory locations that are faster to access.

The LRU (Least Recently Used) Cache discatds the least recently used items first. The algorithm keeps track of;
* what was used 

The [`functools`](https://docs.python.org/3/library/functools.html) module deals with high-order functions:
- functions which operate on 
- returning functions
- other callable objects

The `lru_cache()` helps reduce the execution time of the function by using the memoization technique

Caching in action for the Fibonacci sequence. The least recently used algorithm can cache the return values that are dependent on the arguments that have been passed to the function.

In [ ]:
def fib(n):
    if n < 2:
        return n
    else:
        return fib(n-1) + fib(n-2)

t1 = timeit.Timer("fib(40)", "from __main__ import fib")

print(t1.timeit(1))

<details>
    <summary markdown="span"><b></b></summary>
    Not great is it? So lets add in the <code>lru_cache</code> decorator.
</details>

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=100)
def fib(n):
    if n < 2:
        return n
    else:
        return fib(n-1) + fib(n-2)
    
t1 = timeit.Timer("fib(40)", "from __main__ import fib")

print(t1.timeit(1))

<details>
    <summary markdown="span"><b></b></summary>
    A significant improvement, and only by adding two lines of code!
</details>


***
# <center> [Exercise](./01-Exercise-Fundamentals.ipynb)
****
****